In [1]:
!pip install langid

     |████████████████████████████████| 1.9 MB 36.4 MB/s 
  Created wheel for langid: filename=langid-1.1.6-py3-none-any.whl size=1941187 sha256=b19dcb6b5dc06d1ecf56632232b385752d7a5fdb46d23b5426b70f60f92e47dc
  Stored in directory: /root/.cache/pip/wheels/2b/bb/7f/11e4db39477278161e882eadc46fb558949a28b13470fc74b8
Successfully built langid


In [17]:
from langid import langid
import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [9]:
text = "I am here"
lang = langid.classify(text)
print(lang)

('en', -6.440452575683594)


In [8]:
langid.rank(text)

[('en', -6.440452575683594),
 ('de', -7.602324962615967),
 ('lb', -12.375750064849854),
 ('rw', -12.908110618591309),
 ('ro', -12.978541374206543),
 ('es', -13.397968292236328),
 ('cy', -13.39807653427124),
 ('it', -14.155198097229004),
 ('br', -14.201417922973633),
 ('da', -14.257980346679688),
 ('sq', -14.292641162872314),
 ('pt', -14.425897121429443),
 ('zh', -14.7821626663208),
 ('nl', -14.963902950286865),
 ('fr', -15.118206024169922),
 ('la', -15.191981315612793),
 ('ga', -15.509129524230957),
 ('id', -15.718591690063477),
 ('ku', -16.035820484161377),
 ('ms', -16.112281799316406),
 ('ar', -16.47485589981079),
 ('se', -16.542413115501404),
 ('ht', -16.566608428955078),
 ('af', -16.632877826690674),
 ('lo', -16.72516632080078),
 ('eo', -16.803476810455322),
 ('qu', -16.862179279327393),
 ('et', -17.037346363067627),
 ('is', -17.109008312225342),
 ('tl', -17.202204942703247),
 ('fo', -17.218571662902832),
 ('xh', -17.247032642364502),
 ('sw', -17.37736177444458),
 ('ca', -17.471621

In [14]:
def getLang(s):
  lang = langid.classify(s)  # Returns lang label (e.g. 'en', -4.3590) and probability
  label = lang[0] # returns the last two chars like en or es
  return label


getLang("Je suis")  # English!?!

'en'

In [19]:
text = "This is AMMI, je suis le professeur!"
tokens = word_tokenize(text)
l = []

for i in range(len(tokens)):
  lang_rank = langid.classify(tokens[i])
  label = lang_rank[0]
  l.append(label)

print(l)

['en', 'en', 'en', 'en', 'en', 'en', 'en', 'fr', 'en']


In [20]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [21]:
from tqdm import tqdm

# Two lists to hold language specified in file and another for predicted languages
original_list = []
pred_list = []

with open('/content/drive/MyDrive/lid_spaeng/train.conll') as f:
  # num for number in enumerate and line for each line read
  lines = f.readlines()
  for num, line in tqdm(enumerate(lines), total=len(lines)):
    # Lines that start with #, sent enums and hashtags
    if (line[0] == '#'):
      continue
    elif (line == '\n'):  # skip separator lines too
      continue
    else:
      word, lang = line.split() # split lines to word and language
      if ((lang == 'other') or (lang == 'ambiguous') or (lang == 'unk') or (lang == 'mixed') or (lang == 'fw') or (lang == 'ne')): # Skip tokens with other language tags too
        continue
      else:
        pred_lang = getLang(word) # Use getLang to predict language of word and append it to a list
        pred_list.append(pred_lang)
        original_list.append(lang)  # A list to hold the languages as specified in the file

100%|██████████| 295281/295281 [04:30<00:00, 1091.25it/s]


In [23]:
# Two lists to hold language specified in file and another for predicted languages
devoriginal_list = []
devpred_list = []

with open('/content/drive/MyDrive/lid_spaeng/dev.conll') as dev:
  # num for number in enumerate and line for each line read
  lines = dev.readlines()
  for num, line in tqdm(enumerate(lines), total=len(lines)):
    # Lines that start with #, sent enums and hashtags
    if (line[0] == '#'):
      continue
    elif (line == '\n'):  # skip separator lines too
      continue
    else:
      word, lang = line.split() # split lines to word and language
      if ((lang == 'other') or (lang == 'ambiguous') or (lang == 'unk') or (lang == 'mixed') or (lang == 'fw') or (lang == 'ne')): # Skip tokens with other language tags too: # Skip tokens with other language tags too
        continue
      else:
        pred_lang = getLang(word) # Use Fasttext to predict language of word and append it to a list
        devpred_list.append(pred_lang)
        devoriginal_list.append(lang)  # A list to hold the languages as specified in the file

100%|██████████| 47055/47055 [00:50<00:00, 935.16it/s]


In [24]:
print(len(pred_list))
print(len(original_list))

print(len(devpred_list))
print(len(devoriginal_list))

189265
189265
31405
31405


In [25]:
from sklearn.metrics import f1_score
import numpy as np

f1_train = f1_score(np.array(original_list), np.array(pred_list), average='micro')
print(f1_train)

0.48773941299236523
